## This notebook is a bit messy because it was used as a sandbox to explore the data. It's better if you first have a look at :
- Data e

In [23]:
import sys, os, csv, requests, re, bs4, datetime, time, threading, wget, pdfminer, geopy #geopandas, 
import pandas as pd
import matplotlib.pyplot as plt
from pdfminer.high_level import extract_text
from geopy.geocoders import Nominatim
import numpy as np
from requests import get
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 
import json
from nltk.corpus import stopwords

In [2]:
path='C:/Users/etien/Python/Ireland'
os.chdir(path)
os.getcwd()

'C:\\Users\\etien\\Python\\Ireland'

## Here is a view of what my workspace look like, after scraping PDFs (Reports folder) and converting the PDF to txt file (Reports/Plain text folder):

In [5]:
print('*** in the working directory:\n')
print(os.listdir())
print('\n*** in the Reports directory:')
print('\nhead: ',os.listdir('Reports/')[0:3], '...')
print('... tail: ', os.listdir('Reports/')[-3:])
print('there are:', len(os.listdir('Reports/')[:-1]), 'files')
print('path in PATH_REPORTS')
print('filenames in LIST_REPORTS')
PATH_REPORTS='Reports/'
LIST_REPORTS=os.listdir('Reports/')[:-1]

print('\n\n*** in the Reports/Plain text directory:')
print('\nhead: ',os.listdir('Reports/Plain text/')[0:3], '...')
print('... tail: ', os.listdir('Reports/Plain text/')[-3:])
print('there are:', len(os.listdir('Reports/Plain text/')[:-1]), 'files')
print('path in PATH_PLAIN')
print('filenames in LIST_PLAIN')
PATH_PLAIN='Reports/Plain text/'
LIST_PLAIN=os.listdir('Reports/Plain text')[:-2]

*** in the working directory:

['Ireland_Map.gpkg', 'Reports', 'words_alpha.txt']

*** in the Reports directory:

head:  ['001125B_24_11_2011.pdf', '00467B_15_12_2011.pdf', '00512D_09_06_2011.pdf'] ...
... tail:  ['91550B_13_03_2018.pdf', '91614B_02_12_2020.pdf', 'Plain text']
there are: 2470 files
path in PATH_REPORTS
filenames in LIST_REPORTS


*** in the Reports/Plain text directory:

head:  ['00467B_15_12_2011.txt', '00538V_25_10_2012.txt', '00606M_25_05_2015.txt'] ...
... tail:  ['91614B_02_12_2020.txt', 'filtered', 'report1_19557H_04_12_2014.txt']
there are: 2010 files
path in PATH_PLAIN
filenames in LIST_PLAIN


## First approach: dictionary and counts of word (pure Python)

In [79]:
total_words=set()
path=PATH_PLAIN
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN+fname,'r')
    for line in f.readlines():
        total_words |= set([word.strip().lower() for word in line.split()])
len(total_words)

45598

In [80]:
realwords_in_english=set([word.strip().lower() for word in open('words_alpha.txt','r').readlines()])
len(realwords_in_english)

370103

In [81]:
total_realwords=total_words.intersection(realwords_in_english)
len(total_realwords)

10566

In [82]:
list(total_realwords)[:30]

['configuration',
 'requirements',
 'ais',
 'anam',
 'echoes',
 'functioned',
 'method',
 'satellite',
 'ice',
 'anglican',
 'consciousness',
 'meaningfulness',
 'semantic',
 'sees',
 'factors',
 'arbitrate',
 'employing',
 'emanate',
 'tickets',
 'memo',
 'undiscovered',
 'willingness',
 'performing',
 'explicit',
 'requesting',
 'department',
 'lombard',
 'thinker',
 'awaited',
 'expires']

There are 10566 different words. Let's see there number of occurence in the corpus.

In [35]:
word_counts={}
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN + fname,'r')
    for line in f.readlines():
        for word in [word.strip() for word in line.split()]:
            if word in total_realwords:
                word_counts[word]=word_counts.get(word,0)+1

In [36]:
sorted_word_counts = { k:v for k, v in sorted(word_counts.items(), key=lambda item: item[1], reverse=True) }
for stopword in stopwords.words('english'):
    if stopword in sorted_word_counts:
        del sorted_word_counts[stopword]
sorted_word_counts

{'school': 60382,
 'learning': 37743,
 'good': 26543,
 'pupils': 24355,
 'management': 23609,
 'teaching': 22273,
 'board': 21012,
 'quality': 20247,
 'support': 17797,
 'students': 17088,
 'work': 15148,
 'teachers': 14652,
 'na': 13601,
 'development': 13483,
 'range': 12171,
 'staff': 10989,
 'use': 10701,
 'student': 9986,
 'well': 9923,
 'high': 9809,
 'needs': 9364,
 'planning': 9225,
 'principal': 9120,
 'lessons': 8833,
 'leadership': 8797,
 'effective': 8298,
 'practice': 8229,
 'provided': 8078,
 'parents': 7933,
 'provision': 7818,
 'recommendations': 7812,
 'report': 7547,
 'areas': 7459,
 'team': 7402,
 'also': 7173,
 'ar': 6957,
 'subject': 6780,
 'recommended': 6706,
 'review': 6681,
 'ensure': 6669,
 'assessment': 6636,
 'used': 6572,
 'positive': 6490,
 'senior': 6327,
 'number': 6276,
 'improvement': 6264,
 'pupil': 6230,
 'language': 6041,
 'progress': 5991,
 'inspection': 5802,
 'skills': 5716,
 'overall': 5450,
 'plan': 5413,
 'class': 5354,
 'approach': 5213,
 'go

In [37]:
len(sorted_word_counts)

9719

-> Now we have 9719 words

## Using an API to create a topic-related set of words
I tried to select words indicating a danger, or a building-related issue.

In [39]:
topic = ['danger','risk','harmful','safety','security','heat','gas','cold','building','air','accident','construction','toilet','bathroom','hygiene','sanitation','water','repair']
topic_related_words=set()
for word in topic:
    url = 'https://api.datamuse.com/words?topics='+word
    for element in get(url).json():
        if element['word'] not in stopwords.words('english'):
            topic_related_words |= {element['word']}
len(topic_related_words)

1497

In [43]:
topic_related_words_in_corpus=topic_related_words.intersection(total_realwords)
len(topic_related_words_in_corpus)

470

### Manual assessement of the relevance of these words to the chosen topic

In [45]:
copy=topic_related_words_in_corpus.copy()
for word in list(copy):
    if input('Remove '+ word + '? (press any key then enter. if you want to keep it just press enter)'):
        topic_related_words_in_corpus.remove(word)

Remove welfare? (press any key then enter. if you want to keep it just press enter)y
Remove lir? (press any key then enter. if you want to keep it just press enter)y
Remove hazards? (press any key then enter. if you want to keep it just press enter)
Remove flying? (press any key then enter. if you want to keep it just press enter)y
Remove accident? (press any key then enter. if you want to keep it just press enter)
Remove serious? (press any key then enter. if you want to keep it just press enter)
Remove ice? (press any key then enter. if you want to keep it just press enter)
Remove create? (press any key then enter. if you want to keep it just press enter)y
Remove drink? (press any key then enter. if you want to keep it just press enter)
Remove stream? (press any key then enter. if you want to keep it just press enter)y
Remove safer? (press any key then enter. if you want to keep it just press enter)
Remove vetting? (press any key then enter. if you want to keep it just press enter)y


Remove car? (press any key then enter. if you want to keep it just press enter)
Remove yard? (press any key then enter. if you want to keep it just press enter)
Remove supply? (press any key then enter. if you want to keep it just press enter)y
Remove zeal? (press any key then enter. if you want to keep it just press enter)y
Remove cleaners? (press any key then enter. if you want to keep it just press enter)
Remove land? (press any key then enter. if you want to keep it just press enter)y
Remove property? (press any key then enter. if you want to keep it just press enter)
Remove risk? (press any key then enter. if you want to keep it just press enter)
Remove passion? (press any key then enter. if you want to keep it just press enter)
Remove cleanliness? (press any key then enter. if you want to keep it just press enter)
Remove agua? (press any key then enter. if you want to keep it just press enter)y
Remove beach? (press any key then enter. if you want to keep it just press enter)y
Rem

Remove fish? (press any key then enter. if you want to keep it just press enter)y
Remove prone? (press any key then enter. if you want to keep it just press enter)y
Remove unsafe? (press any key then enter. if you want to keep it just press enter)
Remove foresight? (press any key then enter. if you want to keep it just press enter)y
Remove creating? (press any key then enter. if you want to keep it just press enter)y
Remove recovery? (press any key then enter. if you want to keep it just press enter)
Remove tower? (press any key then enter. if you want to keep it just press enter)y
Remove assured? (press any key then enter. if you want to keep it just press enter)
Remove check? (press any key then enter. if you want to keep it just press enter)y
Remove pressure? (press any key then enter. if you want to keep it just press enter)
Remove bucket? (press any key then enter. if you want to keep it just press enter)
Remove severe? (press any key then enter. if you want to keep it just press 

Remove transport? (press any key then enter. if you want to keep it just press enter)
Remove implementation? (press any key then enter. if you want to keep it just press enter)y
Remove cleaning? (press any key then enter. if you want to keep it just press enter)
Remove weather? (press any key then enter. if you want to keep it just press enter)
Remove properties? (press any key then enter. if you want to keep it just press enter)y
Remove subtle? (press any key then enter. if you want to keep it just press enter)y
Remove reconstruction? (press any key then enter. if you want to keep it just press enter)
Remove continuity? (press any key then enter. if you want to keep it just press enter)y
Remove actual? (press any key then enter. if you want to keep it just press enter)y
Remove management? (press any key then enter. if you want to keep it just press enter)y
Remove real? (press any key then enter. if you want to keep it just press enter)y
Remove remedies? (press any key then enter. if y

Remove flooding? (press any key then enter. if you want to keep it just press enter)
Remove reassurance? (press any key then enter. if you want to keep it just press enter)y
Remove museum? (press any key then enter. if you want to keep it just press enter)y
Remove river? (press any key then enter. if you want to keep it just press enter)y
Remove guarantees? (press any key then enter. if you want to keep it just press enter)
Remove headquarters? (press any key then enter. if you want to keep it just press enter)y
Remove sanitary? (press any key then enter. if you want to keep it just press enter)
Remove crisis? (press any key then enter. if you want to keep it just press enter)
Remove remediation? (press any key then enter. if you want to keep it just press enter)
Remove constructed? (press any key then enter. if you want to keep it just press enter)
Remove service? (press any key then enter. if you want to keep it just press enter)y
Remove steam? (press any key then enter. if you want 

In [46]:
topic_related_words_in_corpus

{'accident',
 'adverse',
 'aerial',
 'air',
 'architect',
 'architectural',
 'architecture',
 'assurance',
 'assured',
 'atmosphere',
 'attack',
 'basin',
 'bathroom',
 'benign',
 'blast',
 'block',
 'breath',
 'bucket',
 'building',
 'buildings',
 'built',
 'car',
 'clean',
 'cleaner',
 'cleaners',
 'cleaning',
 'cleanliness',
 'comfort',
 'compromise',
 'concern',
 'conservation',
 'consolidate',
 'consolidating',
 'consolidation',
 'construct',
 'constructed',
 'constructing',
 'construction',
 'constructions',
 'cool',
 'crisis',
 'damage',
 'danger',
 'dangerous',
 'dangers',
 'defence',
 'depletion',
 'detriment',
 'devastating',
 'difficulty',
 'disruptive',
 'dog',
 'drainage',
 'drains',
 'drink',
 'emergency',
 'engineering',
 'environmental',
 'error',
 'exposure',
 'facility',
 'fear',
 'fire',
 'fixed',
 'flood',
 'flooding',
 'floor',
 'floors',
 'fluid',
 'food',
 'foyer',
 'freeze',
 'gas',
 'guarantee',
 'guaranteed',
 'guarantees',
 'guard',
 'guarding',
 'habitat',
 

In [84]:
removed_topic_1 = {'achievement',
 'acquiring',
 'actual',
 'address',
 'adventure',
 'ages',
 'agua',
 'aire',
 'airport',
 'alley',
 'aquatics',
 'assembling',
 'assessment',
 'assurances',
 'augmentative',
 'augmenting',
 'backed',
 'basis',
 'beach',
 'birth',
 'blocks',
 'bond',
 'born',
 'broadcast',
 'build',
 'cabinet',
 'capital',
 'carbon',
 'care',
 'case',
 'cause',
 'certainty',
 'certificate',
 'challenge',
 'chance',
 'chances',
 'check',
 'chill',
 'compensate',
 'completion',
 'complex',
 'compliance',
 'confidence',
 'confidentiality',
 'consequence',
 'consequences',
 'constructive',
 'consuming',
 'continuity',
 'contractor',
 'correct',
 'correction',
 'council',
 'course',
 'courteousness',
 'coverage',
 'create',
 'creating',
 'creation',
 'cultivating',
 'design',
 'designing',
 'detrimental',
 'develop',
 'developing',
 'development',
 'elaboration',
 'energy',
 'enforcement',
 'enhancing',
 'ensure',
 'ensuring',
 'enthusiasm',
 'equity',
 'erection',
 'essence',
 'establish',
 'establishing',
 'establishment',
 'etiquette',
 'event',
 'expand',
 'expanding',
 'expansion',
 'expression',
 'far',
 'feature',
 'fervour',
 'filling',
 'fish',
 'fluency',
 'flying',
 'foresight',
 'forge',
 'forging',
 'formation',
 'forming',
 'formulation',
 'fostering',
 'foundation',
 'free',
 'fresh',
 'furthering',
 'game',
 'generating',
 'happened',
 'harvest',
 'head',
 'headquarters',
 'heath',
 'heightening',
 'home',
 'house',
 'housing',
 'implantation',
 'implementation',
 'importance',
 'improvement',
 'improving',
 'inactive',
 'increasing',
 'industry',
 'installation',
 'installing',
 'instance',
 'institution',
 'integration',
 'intelligence',
 'intense',
 'intensity',
 'interpretation',
 'john',
 'kit',
 'labour',
 'lake',
 'land',
 'light',
 'likelihood',
 'likely',
 'line',
 'lir',
 'low',
 'making',
 'management',
 'manners',
 'may',
 'melody',
 'might',
 'monitoring',
 'museum',
 'natural',
 'navigable',
 'new',
 'notification',
 'obstacle',
 'occurred',
 'occurrence',
 'occurrences',
 'ocean',
 'odds',
 'open',
 'opportunity',
 'order',
 'overhaul',
 'overhead',
 'oversight',
 'peace',
 'perfect',
 'plane',
 'planning',
 'plot',
 'possibility',
 'potential',
 'potentially',
 'premises',
 'preparation',
 'presentations',
 'production',
 'project',
 'projects',
 'prone',
 'properties',
 'prospect',
 'provision',
 'public',
 'punctuality',
 'quality',
 'question',
 'radio',
 'rationalisation',
 'real',
 'realisation',
 'reassurance',
 'reconstitution',
 'rectify',
 'redress',
 'reform',
 'reformed',
 'reforms',
 'regeneration',
 'reinforcing',
 'reinstate',
 'reinstatement',
 'residence',
 'reverse',
 'rising',
 'river',
 'route',
 'sang',
 'satisfaction',
 'scene',
 'sea',
 'send',
 'service',
 'setting',
 'side',
 'site',
 'sky',
 'species',
 'stabilisation',
 'stability',
 'stake',
 'state',
 'strain',
 'stream',
 'streams',
 'strengthening',
 'student',
 'students',
 'subtle',
 'supply',
 'surface',
 'surprise',
 'susceptible',
 'swimming',
 'terminal',
 'tower',
 'transmit',
 'travel',
 'treatment',
 'tune',
 'twist',
 'unfortunate',
 'unhelpful',
 'upgrade',
 'upgrading',
 'upkeep',
 'upstairs',
 'venture',
 'vetting',
 'viability',
 'warmly',
 'warmth',
 'welfare',
 'wells',
 'work',
 'works',
 'zeal'}

In [47]:
removed = copy - topic_related_words_in_corpus
removed

{'achievement',
 'acquiring',
 'actual',
 'address',
 'adventure',
 'ages',
 'agua',
 'aire',
 'airport',
 'alley',
 'aquatics',
 'assembling',
 'assessment',
 'assurances',
 'augmentative',
 'augmenting',
 'backed',
 'basis',
 'beach',
 'birth',
 'blocks',
 'bond',
 'born',
 'broadcast',
 'build',
 'cabinet',
 'capital',
 'carbon',
 'care',
 'case',
 'cause',
 'certainty',
 'certificate',
 'challenge',
 'chance',
 'chances',
 'check',
 'chill',
 'compensate',
 'completion',
 'complex',
 'compliance',
 'confidence',
 'confidentiality',
 'consequence',
 'consequences',
 'constructive',
 'consuming',
 'continuity',
 'contractor',
 'correct',
 'correction',
 'council',
 'course',
 'courteousness',
 'coverage',
 'create',
 'creating',
 'creation',
 'cultivating',
 'design',
 'designing',
 'detrimental',
 'develop',
 'developing',
 'development',
 'elaboration',
 'energy',
 'enforcement',
 'enhancing',
 'ensure',
 'ensuring',
 'enthusiasm',
 'equity',
 'erection',
 'essence',
 'establish',


In [48]:
print('i removed manually ', len(removed), ' words and i kept', len(topic_related_words_in_corpus),' words.')

i removed manually  258  words and i kept 212  words.


In [50]:
topic_related_word_counts={}
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN + fname,'r')
    for line in f.readlines():
        for word in [word.strip() for word in line.split()]:
            if word in topic_related_words_in_corpus:
                topic_related_word_counts[word]=topic_related_word_counts.get(word,0)+1

In [51]:
sorted_topic_related_word_counts = { k:v for k, v in sorted(topic_related_word_counts.items(), key=lambda item: item[1], reverse=True) }
sorted_topic_related_word_counts

{'building': 1615,
 'atmosphere': 1424,
 'structures': 1348,
 'life': 1044,
 'structure': 992,
 'safe': 970,
 'wellbeing': 895,
 'protection': 860,
 'safety': 659,
 'health': 650,
 'maintain': 571,
 'maintenance': 494,
 'problem': 394,
 'hard': 394,
 'consolidate': 387,
 'room': 287,
 'serious': 277,
 'infrastructure': 258,
 'environmental': 247,
 'consolidation': 246,
 'negative': 243,
 'risk': 239,
 'secure': 224,
 'buildings': 224,
 'space': 213,
 'concern': 210,
 'built': 202,
 'warm': 157,
 'healthy': 154,
 'difficulty': 151,
 'restorative': 122,
 'refurbishment': 119,
 'construction': 105,
 'facility': 95,
 'prevention': 73,
 'consolidating': 69,
 'hall': 68,
 'construct': 67,
 'clean': 52,
 'constructed': 49,
 'restructuring': 49,
 'fire': 48,
 'uncertainty': 47,
 'security': 44,
 'cleaning': 41,
 'traffic': 39,
 'incident': 38,
 'stable': 38,
 'old': 38,
 'incidents': 37,
 'incidence': 36,
 'exposure': 35,
 'integrity': 35,
 'yard': 33,
 'constructing': 33,
 'severe': 32,
 'pre

Second-thought deletions ['building', 'atmosphere', 'structure', 'structure', 'passion', 'buildings']

In [56]:
for element in ['building', 'atmosphere', 'structure', 'structures', 'passion', 'buildings','life']:
    if element in sorted_topic_related_word_counts:
        del sorted_topic_related_word_counts[element]

In [57]:
sorted_topic_related_word_counts

{'safe': 970,
 'wellbeing': 895,
 'protection': 860,
 'safety': 659,
 'health': 650,
 'maintain': 571,
 'maintenance': 494,
 'problem': 394,
 'hard': 394,
 'consolidate': 387,
 'room': 287,
 'serious': 277,
 'infrastructure': 258,
 'environmental': 247,
 'consolidation': 246,
 'negative': 243,
 'risk': 239,
 'secure': 224,
 'space': 213,
 'concern': 210,
 'built': 202,
 'warm': 157,
 'healthy': 154,
 'difficulty': 151,
 'restorative': 122,
 'refurbishment': 119,
 'construction': 105,
 'facility': 95,
 'prevention': 73,
 'consolidating': 69,
 'hall': 68,
 'construct': 67,
 'clean': 52,
 'constructed': 49,
 'restructuring': 49,
 'fire': 48,
 'uncertainty': 47,
 'security': 44,
 'cleaning': 41,
 'traffic': 39,
 'incident': 38,
 'stable': 38,
 'old': 38,
 'incidents': 37,
 'incidence': 36,
 'exposure': 35,
 'integrity': 35,
 'yard': 33,
 'constructing': 33,
 'severe': 32,
 'preparedness': 31,
 'pressure': 27,
 'attack': 27,
 'renovation': 25,
 'risks': 24,
 'cleaner': 22,
 'securing': 22,


## counting topic-related words in each report and creating a JSON file

In [58]:
data_to_visualize=[{'topic':'safety', 'value_by_school':[]}]
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN + fname,'r')
    count=0
    for line in f.readlines():
        for word in [word.strip() for word in line.split()]:
            if word in sorted_topic_related_word_counts:
                count+=1
    data_to_visualize[0]['value_by_school'].append({'school':fname[:6],'value':count})
data_to_visualize

[{'topic': 'safety',
  'value_by_school': [{'school': '00467B', 'value': 3},
   {'school': '00538V', 'value': 8},
   {'school': '00606M', 'value': 2},
   {'school': '00729F', 'value': 3},
   {'school': '00752A', 'value': 2},
   {'school': '00779U', 'value': 0},
   {'school': '00856M', 'value': 4},
   {'school': '00934G', 'value': 6},
   {'school': '00984V', 'value': 5},
   {'school': '01013N', 'value': 3},
   {'school': '01086R', 'value': 1},
   {'school': '01170G', 'value': 8},
   {'school': '01285A', 'value': 5},
   {'school': '01300Q', 'value': 3},
   {'school': '01356U', 'value': 7},
   {'school': '01421F', 'value': 7},
   {'school': '01572D', 'value': 3},
   {'school': '01574H', 'value': 5},
   {'school': '01607T', 'value': 5},
   {'school': '01676P', 'value': 2},
   {'school': '01687U', 'value': 1},
   {'school': '01795A', 'value': 2},
   {'school': '01840C', 'value': 7},
   {'school': '01866U', 'value': 2},
   {'school': '01867W', 'value': 0},
   {'school': '02114B', 'value': 5}

In [60]:
f=open('data_to_visualize.json','w')
json.dump(data_to_visualize,f)

## Searching one word in the corpus

In [119]:
word_to_search = input('Search word : ')
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN + fname,'r')
    count=0
    for line in f.readlines():
        for word in [word.strip() for word in line.split()]:
            if (word == word_to_search):
                count+=1
    if count:
        print(fname, ' : ', count, ' occurence(s).')

Search word : dog
18408I_05_12_2018.txt  :  1  occurence(s).


_Searching the word 'dog' retrieved 1 file :_

"The effective and  flexible  use  of  activities  in  PE,  Art,  Music,  cooking,  along  with  students’  access  to 
services  providing  equine  therapy,  **dog  therapy**  and  other  supports  contribute  to  the 
supportive, learner-centred culture evident in the school.  The whole-school team approach 
to the management of the students is commendable."

## Displaying a (rare) word in its context

In [67]:
def sentences_in_file(fname,word):
    f=open(PATH_PLAIN + fname,'r')
    sentences_list = f.read().split('.')
    sentences_with_word = []
    for sentence in sentences_list:
        if word in sentence:
            sentences_with_word.append(sentence)
    return sentences_with_word

fname=input('File name : ')
word=input('Word : ')
for sentence in sentences_in_file(fname,word):
    print(sentence)

File name : 18408I_05_12_2018.txt
Word : dog
  The effective and flexible use of activities in PE, Art, Music, cooking, along with students’ access to services providing equine therapy, dog therapy and other supports contribute to the supportive, learner-centred culture evident in the school


In [73]:
def sentences_in_corpus(word):
    sentences_by_file={}
    for fname in LIST_PLAIN:
        sentences_with_word=sentences_in_file(fname,word)
        if sentences_with_word:
            sentences_by_file[fname]=sentences_with_word
    return sentences_by_file

word=input('Word : ')
sentences_in_corpus=sentences_in_corpus(word)
for fname in sentences_in_corpus:
    print('\nIn file ' + fname + ':')
    for sentence in sentences_in_corpus[fname]:
        print(sentence)

Word : fish

In file 65582Q_20_06_2017.txt:
  Some assessment for learning (AfL) strategies were employed in lessons, including mini whiteboards, fishbone diagrams, teacher circulation

In file 76083J_14_06_2016.txt:
 Other AfL strategies, such as the use of fish bone diagrams, traffic lights 
and mini-white boards, were used to good effect

In file 91431Q_10_09_2019.txt:
 It is positive to note there is an advisory board of studies currently investigating the viability of one-hour lessons and considering JC short courses to meet the specific needs of students in areas such as fisheries and farming


In [74]:
def sentences_in_corpus(word):
    sentences_by_file={}
    for fname in LIST_PLAIN:
        sentences_with_word=sentences_in_file(fname,word)
        if sentences_with_word:
            sentences_by_file[fname]=sentences_with_word
    return sentences_by_file

word=input('Word : ')
sentences_in_corpus=sentences_in_corpus(word)
for fname in sentences_in_corpus:
    print('\nIn file ' + fname + ':')
    for sentence in sentences_in_corpus[fname]:
        print(sentence)

Word : farm

In file 19404F_09_02_2012.txt:
 In order to develop pupils’ confidence, very interesting individual 
projects are undertaken, including a notable project by a pupil on farm machinery

In file 64150F_18_06_2014.txt:
3  Management of facilities 

The  school  grounds  are  set  within  the  extensive  parkland,  gardens  and  farmland  of  the 
original nineteenth century baronial castle that is the central focus of the monastery and is 
linked to the new school building

In file 72290R_26_05_2014.txt:
  Among  other 
priorities identified by the board are developing links with local third level providers and 
maximising the potential created by the school’s facilities, in particular its working farm, 
for the development of curriculum-related activities and programmes

In file 91431Q_10_09_2019.txt:
 It is positive to note there is an advisory board of studies currently investigating the viability of one-hour lessons and considering JC short courses to meet the specific nee

In [75]:
def sentences_in_corpus(word):
    sentences_by_file={}
    for fname in LIST_PLAIN:
        sentences_with_word=sentences_in_file(fname,word)
        if sentences_with_word:
            sentences_by_file[fname]=sentences_with_word
    return sentences_by_file

word=input('Word : ')
sentences_in_corpus=sentences_in_corpus(word)
for fname in sentences_in_corpus:
    print('\nIn file ' + fname + ':')
    for sentence in sentences_in_corpus[fname]:
        print(sentence)

Word : horse

In file 19108B_19_06_2018.txt:
 Pupils enjoy regular 
opportunities  in  Physical  Education,  including  gross  motor  activities,  movement  to  music, 
swimming and horse-riding

In file 19724A_31_05_2018.txt:
 Pupils also utilise external facilities to access horse riding, swimming, and bowling, and exercise at a local gym


## Shifting to sport-related words

In [78]:
topic = ['sports','animals','football','hockey']
topic_related_words=set()
for word in topic:
    url = 'https://api.datamuse.com/words?topics='+word
    for element in get(url).json():
        if element['word'] not in stopwords.words('english'):
            topic_related_words |= {element['word']}
len(topic_related_words)

335

In [83]:
topic_related_words_in_corpus=topic_related_words.intersection(total_realwords)
len(topic_related_words_in_corpus)

68

### Manual assessement of the relevance of these words to the chosen topic

In [85]:
copy2=topic_related_words_in_corpus.copy()
for word in list(copy2):
    if input('Remove '+ word + '? (press any key then enter. Or press enter to pass)'):
        topic_related_words_in_corpus.remove(word)

Remove fun? (press any key then enter. Or press enter to pass)
Remove cup? (press any key then enter. Or press enter to pass)y
Remove team? (press any key then enter. Or press enter to pass)y
Remove events? (press any key then enter. Or press enter to pass)y
Remove campaign? (press any key then enter. Or press enter to pass)y
Remove dogs? (press any key then enter. Or press enter to pass)
Remove football? (press any key then enter. Or press enter to pass)
Remove athletics? (press any key then enter. Or press enter to pass)
Remove handball? (press any key then enter. Or press enter to pass)
Remove ice? (press any key then enter. Or press enter to pass)
Remove equestrian? (press any key then enter. Or press enter to pass)
Remove disciplines? (press any key then enter. Or press enter to pass)y
Remove play? (press any key then enter. Or press enter to pass)y
Remove famous? (press any key then enter. Or press enter to pass)y
Remove gym? (press any key then enter. Or press enter to pass)
Rem

In [86]:
topic_related_words_in_corpus

{'aquatics',
 'athletic',
 'athletics',
 'badminton',
 'ball',
 'basketball',
 'bears',
 'club',
 'coaches',
 'competitions',
 'cricket',
 'cycling',
 'dogs',
 'equestrian',
 'foot',
 'football',
 'fun',
 'game',
 'games',
 'golf',
 'gym',
 'gymnastics',
 'handball',
 'hockey',
 'ice',
 'league',
 'leisure',
 'match',
 'pet',
 'physical',
 'player',
 'players',
 'playing',
 'recreation',
 'recreational',
 'rugby',
 'soccer',
 'sport',
 'sports',
 'swimming',
 'teams',
 'tennis',
 'veterinary',
 'wildlife'}

In [87]:
removed2 = copy2 - topic_related_words_in_corpus
removed2

{'activities',
 'adventure',
 'body',
 'bullet',
 'campaign',
 'campaigns',
 'codes',
 'cup',
 'disciplines',
 'events',
 'exercise',
 'exercises',
 'famous',
 'feature',
 'feed',
 'flora',
 'heads',
 'individuals',
 'mathematical',
 'movement',
 'play',
 'species',
 'stick',
 'team'}

In [88]:
print('i removed manually ', len(removed2), ' words and i kept', len(topic_related_words_in_corpus),' words.')

i removed manually  24  words and i kept 44  words.


In [89]:
topic_related_word_counts={}
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN + fname,'r')
    for line in f.readlines():
        for word in [word.strip() for word in line.split()]:
            if word in topic_related_words_in_corpus:
                topic_related_word_counts[word]=topic_related_word_counts.get(word,0)+1

In [90]:
sorted_topic_related_word_counts = { k:v for k, v in sorted(topic_related_word_counts.items(), key=lambda item: item[1], reverse=True) }
sorted_topic_related_word_counts

{'physical': 519,
 'teams': 495,
 'sports': 299,
 'games': 238,
 'club': 128,
 'playing': 117,
 'competitions': 82,
 'match': 81,
 'recreational': 57,
 'sport': 56,
 'recreation': 45,
 'swimming': 38,
 'foot': 22,
 'football': 21,
 'ball': 20,
 'fun': 17,
 'basketball': 16,
 'rugby': 12,
 'leisure': 12,
 'gym': 12,
 'soccer': 10,
 'athletics': 9,
 'gymnastics': 9,
 'players': 9,
 'tennis': 8,
 'bears': 7,
 'coaches': 6,
 'hockey': 6,
 'game': 3,
 'handball': 2,
 'wildlife': 2,
 'player': 2,
 'dogs': 1,
 'cycling': 1,
 'aquatics': 1,
 'cricket': 1,
 'equestrian': 1,
 'league': 1,
 'pet': 1,
 'badminton': 1}

Second-thought deletions

In [109]:
for element in ['pet','dogs','bears','wildlife','foot','physical','fun','recreational','recreation']:
    if element in sorted_topic_related_word_counts:
        del sorted_topic_related_word_counts[element]

In [117]:
sports_collection=list(sorted_topic_related_word_counts)
sports_collection.extend(['table tennis','yoga','hurling','aquatics','Gaelic games','Gaelic football','tag-rugby'])
sports_collection

['teams',
 'sports',
 'games',
 'club',
 'playing',
 'competitions',
 'match',
 'sport',
 'swimming',
 'football',
 'ball',
 'basketball',
 'rugby',
 'leisure',
 'gym',
 'soccer',
 'athletics',
 'gymnastics',
 'players',
 'tennis',
 'coaches',
 'hockey',
 'game',
 'handball',
 'player',
 'cycling',
 'aquatics',
 'cricket',
 'equestrian',
 'league',
 'badminton',
 'table tennis',
 'yoga',
 'hurling',
 'aquatics',
 'Gaelic games',
 'Gaelic football',
 'tag-rugby']

In [107]:
data_to_visualize0=[data_to_visualize[0]]
data_to_visualize1=[data_to_visualize[1]]

In [98]:
data_to_visualize.append({'topic':'sports', 'value_by_school':[]})
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN + fname,'r')
    count=0
    for line in f.readlines():
        for word in [word.strip() for word in line.split()]:
            if word in sorted_topic_related_word_counts:
                count+=1
    data_to_visualize[-1]['value_by_school'].append({'school':fname[:6],'value':count})
data_to_visualize

[{'topic': 'safety',
  'value_by_school': [{'school': '00467B', 'value': 3},
   {'school': '00538V', 'value': 8},
   {'school': '00606M', 'value': 2},
   {'school': '00729F', 'value': 3},
   {'school': '00752A', 'value': 2},
   {'school': '00779U', 'value': 0},
   {'school': '00856M', 'value': 4},
   {'school': '00934G', 'value': 6},
   {'school': '00984V', 'value': 5},
   {'school': '01013N', 'value': 3},
   {'school': '01086R', 'value': 1},
   {'school': '01170G', 'value': 8},
   {'school': '01285A', 'value': 5},
   {'school': '01300Q', 'value': 3},
   {'school': '01356U', 'value': 7},
   {'school': '01421F', 'value': 7},
   {'school': '01572D', 'value': 3},
   {'school': '01574H', 'value': 5},
   {'school': '01607T', 'value': 5},
   {'school': '01676P', 'value': 2},
   {'school': '01687U', 'value': 1},
   {'school': '01795A', 'value': 2},
   {'school': '01840C', 'value': 7},
   {'school': '01866U', 'value': 2},
   {'school': '01867W', 'value': 0},
   {'school': '02114B', 'value': 5}

In [111]:
#f=open('data_to_visualize1.json','w')
#json.dump(data_to_visualize1,f)
f=open('data_to_visualize0.json','w')
json.dump(data_to_visualize0,f)

In [106]:
import sys
print(sys.getsizeof(data_to_visualize))
data_to_visualize

104


[{'topic': 'safety',
  'value_by_school': [{'school': '00467B', 'value': 3},
   {'school': '00538V', 'value': 8},
   {'school': '00606M', 'value': 2},
   {'school': '00729F', 'value': 3},
   {'school': '00752A', 'value': 2},
   {'school': '00779U', 'value': 0},
   {'school': '00856M', 'value': 4},
   {'school': '00934G', 'value': 6},
   {'school': '00984V', 'value': 5},
   {'school': '01013N', 'value': 3},
   {'school': '01086R', 'value': 1},
   {'school': '01170G', 'value': 8},
   {'school': '01285A', 'value': 5},
   {'school': '01300Q', 'value': 3},
   {'school': '01356U', 'value': 7},
   {'school': '01421F', 'value': 7},
   {'school': '01572D', 'value': 3},
   {'school': '01574H', 'value': 5},
   {'school': '01607T', 'value': 5},
   {'school': '01676P', 'value': 2},
   {'school': '01687U', 'value': 1},
   {'school': '01795A', 'value': 2},
   {'school': '01840C', 'value': 7},
   {'school': '01866U', 'value': 2},
   {'school': '01867W', 'value': 0},
   {'school': '02114B', 'value': 5}

In [100]:
def sentences_in_corpus(word):
    sentences_by_file={}
    for fname in LIST_PLAIN:
        sentences_with_word=sentences_in_file(fname,word)
        if sentences_with_word:
            sentences_by_file[fname]=sentences_with_word
    return sentences_by_file

word=input('Word : ')
sentences_in_corpus=sentences_in_corpus(word)
for fname in sentences_in_corpus:
    print('\nIn file ' + fname + ':')
    for sentence in sentences_in_corpus[fname]:
        print(sentence)

Word : athletic

In file 01607T_15_04_2014.txt:
 

 

External  coaches  provide  tuition  in  a  range  of  activities  such  as  athletics,  table  tennis, 
soccer and rugby under the direction of the class teachers

In file 01676P_25_11_2016.txt:
 

 

  The  practice  whereby  parents  are  required  to  pay  external  tutors  for  lessons  in 
gymnastics,  yoga  and  athletics  which  take  place  during  the  school  day  contravenes 
Departmental  policy

In file 04075E_29_05_2015.txt:
  The  pupils’  creativity 
across the curriculum is celebrated through excellent displays of their work and their 
achievements are recognised and applauded at fortnightly assembly Participation in a 
range  of  co-curricular  activities  including  the  school  band,  athletics  and  games 
provides  a  particularly  positive  experience  for  the  pupils

In file 08342V_27_03_2012.txt:
 
Parents  and  teachers  have  co-operated  successfully  in  encouraging  pupils’  interest  in 
Mathematics

In [115]:
non_sports_activities_collection=['chess', 'drama', 'choral', 'choral singing', 'music', 'quizzes', 'concerts', 'games', 'field  trips', 'poetry','rhymes', 'song', 'relaxation','creativity', 'self-expression','school  band','playing musical instruments','Irish  cultural  events','Irish club', 'Irish dancing', 'wellbeing committee','reading challenges', 'mathletics', 'mathematics for fun', 'welcome meetings for new parents',
'cultural outings']

In [ ]:
positive_collection=[' fun and shared enjoyment','very inclusive','varied, rich, fun and meaningful activities','particular strength','fun opportunities','derive benefit','fun and supportive environment','celebrated','excellent','particularly  positive  experience','achievements','applauded','recognised','enthusiastically','satisfactorily','successfully','engagement','encouraging','interest','co-operate','vehicle to encourage',]
initiatives_collection=['Initiatives','contribute','co-operate',]
skills_collection=['social skills', 'language skills', 'communication skills', 'creativity','positive attitude', 'inclusive attitude']

In [102]:
def sentences_in_corpus(word):
    sentences_by_file={}
    for fname in LIST_PLAIN:
        sentences_with_word=sentences_in_file(fname,word)
        if sentences_with_word:
            sentences_by_file[fname]=sentences_with_word
    return sentences_by_file

word=input('Word : ')
sentences_in_corpus=sentences_in_corpus(word)
for fname in sentences_in_corpus:
    print('\nIn file ' + fname + ':')
    for sentence in sentences_in_corpus[fname]:
        print(sentence)

Word : fun 

In file 13512B_26_09_2016.txt:
 It would also be worthwhile providing an extended range of fun 
opportunities  for  pupils  to  learn  the  language

In file 16044J_04_09_2015.txt:
  There  is  evidence  that 
pupils in the junior room are developing social, language and communication skills in 
a fun and supportive environment

In file 16573N_26_10_2012.txt:
  The 
promotion of a positive, inclusive attitude to sport and the balance of competitive and fun 
activities  are  commended

In file 18033O_20_12_2019.txt:
   The quality of learning experiences provided to the pupils is a particular strength of the school: pupils engage in varied, rich, fun and meaningful activities

In file 18178R_08_05_2015.txt:
  Pupils  derive  benefit  and 
fun from poetry activities and songs

In file 18506I_06_01_2015.txt:
 
•  School reports have been updated in line with circular 56/11 & 45/14 
•  Maths for fun has been implemented and all classes avail of it 

 
In process 
SSE: SIP NUME

In [103]:
def sentences_in_corpus(word):
    sentences_by_file={}
    for fname in LIST_PLAIN:
        sentences_with_word=sentences_in_file(fname,word)
        if sentences_with_word:
            sentences_by_file[fname]=sentences_with_word
    return sentences_by_file

word=input('Word : ')
sentences_in_corpus=sentences_in_corpus(word)
for fname in sentences_in_corpus:
    print('\nIn file ' + fname + ':')
    for sentence in sentences_in_corpus[fname]:
        print(sentence)

Word : playing

In file 01676P_25_11_2016.txt:
 These involved good 
quality  performance  of  song  singing  and  instrument  playing

In file 02418V_19_01_2012.txt:
  

 
•  Standards  in  Mathematics  are  good  with  pupils  displaying  appropriate  understanding  of 
concepts and their application

In file 02670I_06_12_2017.txt:
  Pupils demonstrate an understanding of musical concepts through listening, song singing and playing instruments

In file 03607G_19_06_2015.txt:
 

the  work  of 

•  During breaks the pupils benefit from the availability of a new all-weather playing area 
and playground
  These 
facilities,  which  include  a  car  park,  a  playground  and  an  all-weather  playing  pitch 
were  developed  in  partnership  with  Leitrim  Sports  and  Recreation  Development 
Group

In file 04067F_01_10_2012.txt:
  Our 
three class room and resource room extension is currently under way and the remainder is used 
as a playing field

In file 05630L_12_04_2019.txt:
 Their 

In [110]:
def sentences_in_corpus(word):
    sentences_by_file={}
    for fname in LIST_PLAIN:
        sentences_with_word=sentences_in_file(fname,word)
        if sentences_with_word:
            sentences_by_file[fname]=sentences_with_word
    return sentences_by_file

word=input('Word : ')
sentences_in_corpus=sentences_in_corpus(word)
for fname in sentences_in_corpus:
    print('\nIn file ' + fname + ':')
    for sentence in sentences_in_corpus[fname]:
        print(sentence)

Word : game

In file 00538V_25_10_2012.txt:
 
Activity  learning  methods  and  mathematical  games  are  used

In file 00934G_27_06_2014.txt:
  Most  lessons  were  presented  effectively  through 
active  and  discovery  learning  approaches,  collaborative  learning  activities,  and  use  of 
ICT  and  games

In file 01607T_15_04_2014.txt:
 Teachers use role-play, pair work 
and  games  effectively  in  teaching
 
Pupils use games, concrete materials and information and communications technology in 
their  learning
  Teachers  employ  a  range  of 
suitable  strategies  such  as  improvisation,  mime,  circle  activities  and  games  in  lessons
  A  wide  range  of 
resources  and  games  is  devised  to  support  pupils’  learning

In file 01795A_12_05_2012.txt:
 Effective use is made 
of language games and enjoyable activities to promote children’s interest
 Props, classroom space and drama games are used  well to enable  pupils to 
enter  into  role  and  to  co-operate  with  

In [ ]:
computer_collection=['computer-assisted learning']
teaching_collection=['role-play','pair work','active  and  discovery  learning  approaches','collaborative  learning  activities','use of ICT','improvisation,  mime,  circle  activities  and  games  in  lessons']

In [116]:
def sentences_in_corpus(word):
    sentences_by_file={}
    for fname in LIST_PLAIN:
        sentences_with_word=sentences_in_file(fname,word)
        if sentences_with_word:
            sentences_by_file[fname]=sentences_with_word
    return sentences_by_file

word=input('Word : ')
sentences_in_corpus=sentences_in_corpus(word)
for fname in sentences_in_corpus:
    print('\nIn file ' + fname + ':')
    for sentence in sentences_in_corpus[fname]:
        print(sentence)

Word : Irish

In file 00467B_15_12_2011.txt:
  The  school  should 
continue  to  allocate  funds  in  support  of  the  purchase  of  books  and  other  reading 
materials in both English and Irish
  
 
Many examples of good practice in the teaching and learning of Irish were noted
 It would be worthwhile to draw up a strategic 
plan for Irish containing specific objectives and to link this work to staff development and 
curriculum  leadership

In file 00538V_25_10_2012.txt:
     Introduction  
The  Convent  School  in  Daingean  Uí  Chúis  is  an  all-Irish  school,  under  the  patronage  of  the 
Bishop of Kerry
 
 
  The standard of learning of Irish is good and creditable progress is noted in the standard 

of oral competence among the majority of the pupils
 
 
It  is  evident  that  the  school  staff  are  very  keen  on  Irish  and  that  they  foster  goodwill 
towards the language and a liking for it, among the pupils
 Irish is taught effectively and 
pleasantly in the scho

  Current priorities include oral language development in Irish and the implementation of the PLC
 Specific targeted actions are outlined in the school improvement plan to support teachers’ approaches and pupils’ learning in Irish
 Process writing in Irish will be reviewed and a progression plan put in place to ensure continuity and development through all classes

In file 16140F_04_10_2012.txt:
 In  Irish, it is recommended 
that  the  pupils’  language  skills  be  developed  more  systematically  through  the  school 
and that their communicative abilities be fostered further
  In  most  areas  of  the  curriculum,  there  are  plentiful  resources  to  support 
teaching and learning, although increased provision of reading material for class libraries, 
in  both  English  and  Irish,  is  advised
  
 
The standard of teaching and learning in Irish varies through the school

In file 16143L_12_12_2017.txt:
   SUMMARY OF MAIN FINDINGS AND RECOMMENDATIONS:  FINDINGS  The overall qualit

 

(cid:1) 

•  Effective  whole-class  teaching  ensures  that  pupils  acquire  a  very  good  range  of 
vocabulary  and  correct  sentence  structures  in  Irish

In file 18098T_26_01_2016.txt:
  Overall,  pupils  exhibit  good  ability  in  speaking  Irish,  particularly  in  their  use  of 
individual words, but there  is  scope for further development  of  their skills  in  constructing full 
sentences
 In certain learning contexts in Irish and with reference to specific pupils, more 
emphasis should be placed on learning new vocabulary in the context of sentences, on pair 
work and on informal conversation
  
 
The quality of learning in Irish is good
 In certain learning contexts and with reference to specific pupils, additional 
emphasis should be placed on learning new words in the context of sentences, on pair work 
and on informal Irish

In file 18101F_08_10_2012.txt:
     Summary of Findings and Recommendations for Further Development 
 
 
The following are the main stren

 

•  Pupils’ communicative ability in Irish is of a very high standard
 The results of standardised tests indicate that pupils 
have made very good progress in Irish, English, and Mathematics because of this work

In file 19780K_30_09_2014.txt:
  In  Irish,  additional  opportunities  should  be  provided 
for  pupils  to  practice  their  oral  skills  in  a  variety  of  contexts  and  closer  monitoring 
should be conducted of their progress
  
 

•  Overall,  pupils  have  a  good  standard  of  learning  in  Irish
  To  enhance  pupils’ 
confidence  and  speaking  skills  it  is  recommended  that  a  stronger  whole-school 
emphasis be placed on the informal use of Irish and additional opportunities provided 
for pupils to practise the language in a variety of contexts
  Pupils  in  some  classes  perform  a  range  of  Irish  and 
English songs to a very high standard

In file 19784S_28_04_2015.txt:
  
 
The standard of teaching, learning and pupil achievement in Irish is good


 Another feature is that the Study Skills lessons for students with exemptions from 
Irish  are  essentially  study  periods  rather  than  formal  lessons
  Due  to  timetabling  constraints  recommendations  such  as  the  introduction  of  a 
taster  programme  for  modern  languages  and  an  increased  amount  of time  for  Irish  in  the 
junior cycle have not been progressed

In file 60291D_13_03_2019.txt:
 Currently students receive support through withdrawal from Irish lessons or non-exam subjects such as Social, Personal and Health Education (SPHE), Physical Education (PE), and Pastoral Care
 The recommendation from the Gaeilge inspection regarding the greater promotion of oral Irish has not been inadequately addressed

In file 60310E_03_05_2019.txt:
 A number of the withdrawal groups include students who do not have identified learning needs, but who are exempt from Irish

In file 60343T_21_06_2018.txt:
WHOLE-SCHOOL EVALUATION – MANAGEMENT, LEADERSHIP AND LEARNING  Dates of 

In [ ]:
language_collection=['learning of Irish', 'all-Irish school', 'oral Irish', 'Irish books','Irish reading','oral skills in Irish','Irish lessons', 'teaching of Irish','learning in Irish', 'competence in Irish', 'ability in Irish','communicative competencies in Irish', 'reading materials in Irish', 'rich Irish']

In [118]:
def sentences_in_corpus(word):
    sentences_by_file={}
    for fname in LIST_PLAIN:
        sentences_with_word=sentences_in_file(fname,word)
        if sentences_with_word:
            sentences_by_file[fname]=sentences_with_word
    return sentences_by_file

word=input('Word : ')
sentences_in_corpus=sentences_in_corpus(word)
for fname in sentences_in_corpus:
    print('\nIn file ' + fname + ':')
    for sentence in sentences_in_corpus[fname]:
        print(sentence)

Word : computer

In file 01572D_02_12_2014.txt:
  In  questionnaires,  administered 
as part of this evaluation, the majority of pupils agreed that they enjoy their learning 
and almost all agreed that they use computers in lessons

In file 01574H_22_09_2014.txt:
 There was greater 
variation  in  their  responses  in  respect  of  the  enjoyment  of  lessons,  the  use  of 
computers during lessons  and participation  in group  work; this variation in response 
should prompt discussion and follow-up at school level

In file 02418V_19_01_2012.txt:
  Pupils  also  indicated  that 
opportunities  such  as  the  use  of  computers  or  engaging  in  group  work  are  not  regular 
features of their learning experience

In file 02428B_11_12_2014.txt:
 A suitable 
programme of work is followed, and pupils have good opportunities to work in pairs and 
in  groups,  to  undertake  mathematical  activities,  to  play  mathematical  games,  to  use 
concrete materials and to engage in computer-a

In [ ]:
'use computers in lessons','computer-based learning environments','ICT lessons','computer room','use  of computers during lessons','computer-assisted learning','shared  computer  drive','computer resources to  present  lessons','use computers','laptop computer games','Computer technology has been provided in all classrooms',
'high usage of computers','computer-assisted  learning','computer  facilitie','computer hardware','computer resources','work with computers','use of ICT','interactive whiteboard','computerised  white-boards','interactive  white boards','information  and communication  technology'

In [ ]:
'The school should increase the pupils’ hands-on access to computers',
'decision  to  introduce  tablet computers across the school'
'In the questionnaires,  the majority  of pupils indicate that they do not use  computers regularly  in  their  learning  and  no  examples  of  pupils  using  ICT  were  evident  during the evaluation'

## Searching one word in the corpus

In [120]:
word_to_search = input('Search word : ')
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN + fname,'r')
    count=0
    for line in f.readlines():
        for word in [word.strip() for word in line.split()]:
            if (word == word_to_search):
                count+=1
    if count:
        print(fname, ' : ', count, ' occurence(s).')

Search word : computer
05754G_15_10_2015.txt  :  1  occurence(s).
06539H_24_10_2014.txt  :  1  occurence(s).
07117J_05_04_2016.txt  :  1  occurence(s).
07481H_18_03_2014.txt  :  1  occurence(s).
08393P_13_03_2014.txt  :  1  occurence(s).
08791E_12_05_2012.txt  :  1  occurence(s).
15700M_15_04_2015.txt  :  1  occurence(s).
16646O_08_04_2014.txt  :  1  occurence(s).
16715H_23_06_2014.txt  :  2  occurence(s).
16767D_23_10_2012.txt  :  2  occurence(s).
17053P_24_02_2015.txt  :  1  occurence(s).
17353E_13_11_2014.txt  :  1  occurence(s).
17440W_27_03_2012.txt  :  1  occurence(s).
17533G_01_03_2012.txt  :  1  occurence(s).
17674B_03_03_2016.txt  :  1  occurence(s).
17847G_29_04_2015.txt  :  1  occurence(s).
17868O_31_03_2014.txt  :  1  occurence(s).
17937H_27_03_2012.txt  :  1  occurence(s).
18028V_15_12_2011.txt  :  1  occurence(s).
18128C_19_01_2015.txt  :  1  occurence(s).
18295V_06_07_2012.txt  :  1  occurence(s).
18346M_05_07_2012.txt  :  1  occurence(s).
18408I_05_12_2018.txt  :  1  oc

In [121]:
negative = list({'concern',
 'deficiencies',
 'difficulties',
 'difficulty',
 'fair',
 'improve',
 'improvements',
 'ineffective',
 'insufficient',
 'poor',
 'serious',
 'shortcomings',
 'unsatisfactory',
 'weak',
 'weaknesses'})

In [123]:
negative_word_counts={}
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN + fname,'r')
    for line in f.readlines():
        for word in [word.strip() for word in line.split()]:
            if word in negative:
                negative_word_counts[word]=negative_word_counts.get(word,0)+1

In [124]:
data_to_visualize=[{'topic':'negative', 'value_by_school':[]}]
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN + fname,'r')
    count=0
    for line in f.readlines():
        for word in [word.strip() for word in line.split()]:
            if word in negative :
                count+=1
    data_to_visualize[0]['value_by_school'].append({'school':fname[:6],'value':count})
data_to_visualize

[{'topic': 'negative',
  'value_by_school': [{'school': '00467B', 'value': 3},
   {'school': '00538V', 'value': 1},
   {'school': '00606M', 'value': 3},
   {'school': '00729F', 'value': 3},
   {'school': '00752A', 'value': 1},
   {'school': '00779U', 'value': 0},
   {'school': '00856M', 'value': 1},
   {'school': '00934G', 'value': 3},
   {'school': '00984V', 'value': 2},
   {'school': '01013N', 'value': 2},
   {'school': '01086R', 'value': 2},
   {'school': '01170G', 'value': 4},
   {'school': '01285A', 'value': 3},
   {'school': '01300Q', 'value': 2},
   {'school': '01356U', 'value': 2},
   {'school': '01421F', 'value': 9},
   {'school': '01572D', 'value': 2},
   {'school': '01574H', 'value': 4},
   {'school': '01607T', 'value': 5},
   {'school': '01676P', 'value': 4},
   {'school': '01687U', 'value': 0},
   {'school': '01795A', 'value': 2},
   {'school': '01840C', 'value': 1},
   {'school': '01866U', 'value': 2},
   {'school': '01867W', 'value': 0},
   {'school': '02114B', 'value': 

In [125]:
topic = ['computer']
topic_related_words=set()
for word in topic:
    url = 'https://api.datamuse.com/words?topics='+word
    for element in get(url).json():
        if element['word'] not in stopwords.words('english'):
            topic_related_words |= {element['word']}
len(topic_related_words)

100

In [126]:
total_words=set()
path=PATH_PLAIN
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN+fname,'r')
    for line in f.readlines():
        total_words |= set([word.strip().lower() for word in line.split()])
realwords_in_english=set([word.strip().lower() for word in open('words_alpha.txt','r').readlines()])
total_realwords=total_words.intersection(realwords_in_english)

In [127]:
topic_related_words_in_corpus = topic_related_words.intersection(total_realwords)
len(topic_related_words_in_corpus)

46

In [128]:
copy = topic_related_words_in_corpus.copy()
for word in list(copy):
    if input('Keep '+ word + '? Press Enter to keep. Press any key + Enter to remove : '):
        topic_related_words_in_corpus.remove(word)

Keep gear? Press Enter to keep. Press any key + Enter to remove : n
Keep internet? Press Enter to keep. Press any key + Enter to remove : 
Keep team? Press Enter to keep. Press any key + Enter to remove : n
Keep integrated? Press Enter to keep. Press any key + Enter to remove : n
Keep calculators? Press Enter to keep. Press any key + Enter to remove : 
Keep computing? Press Enter to keep. Press any key + Enter to remove : 
Keep device? Press Enter to keep. Press any key + Enter to remove : n
Keep laptop? Press Enter to keep. Press any key + Enter to remove : 
Keep terminal? Press Enter to keep. Press any key + Enter to remove : n
Keep software? Press Enter to keep. Press any key + Enter to remove : 
Keep assisted? Press Enter to keep. Press any key + Enter to remove : n
Keep eased? Press Enter to keep. Press any key + Enter to remove : n
Keep video? Press Enter to keep. Press any key + Enter to remove : 
Keep aided? Press Enter to keep. Press any key + Enter to remove : n
Keep pool? Pr

In [129]:
topic_related_words_in_corpus

{'calculation',
 'calculator',
 'calculators',
 'computation',
 'computational',
 'computing',
 'desktop',
 'digital',
 'electronic',
 'hardware',
 'internet',
 'keyboard',
 'laptop',
 'machines',
 'screen',
 'software',
 'tablet',
 'video'}

In [130]:
computer_collection=['computer','computers']+list(topic_related_words_in_corpus)

In [131]:
computer_collection

['computer',
 'computers',
 'internet',
 'calculators',
 'computing',
 'laptop',
 'software',
 'video',
 'machines',
 'desktop',
 'hardware',
 'electronic',
 'keyboard',
 'calculation',
 'tablet',
 'calculator',
 'digital',
 'screen',
 'computational',
 'computation']

In [132]:
safety_collection = {'accident',
 'adverse',
 'aerial',
 'air',
 'architect',
 'architectural',
 'architecture',
 'assurance',
 'assured',
 'atmosphere',
 'attack',
 'basin',
 'bathroom',
 'benign',
 'blast',
 'block',
 'breath',
 'bucket',
 'building',
 'buildings',
 'built',
 'car',
 'clean',
 'cleaner',
 'cleaners',
 'cleaning',
 'cleanliness',
 'comfort',
 'compromise',
 'concern',
 'conservation',
 'consolidate',
 'consolidating',
 'consolidation',
 'construct',
 'constructed',
 'constructing',
 'construction',
 'constructions',
 'cool',
 'crisis',
 'damage',
 'danger',
 'dangerous',
 'dangers',
 'defence',
 'depletion',
 'detriment',
 'devastating',
 'difficulty',
 'disruptive',
 'dog',
 'drainage',
 'drains',
 'drink',
 'emergency',
 'engineering',
 'environmental',
 'error',
 'exposure',
 'facility',
 'fear',
 'fire',
 'fixed',
 'flood',
 'flooding',
 'floor',
 'floors',
 'fluid',
 'food',
 'foyer',
 'freeze',
 'gas',
 'guarantee',
 'guaranteed',
 'guarantees',
 'guard',
 'guarding',
 'habitat',
 'hall',
 'hard',
 'harm',
 'hazard',
 'hazardous',
 'hazards',
 'health',
 'healthcare',
 'healthy',
 'hot',
 'hygiene',
 'hygienic',
 'ice',
 'ill',
 'incidence',
 'incident',
 'incidents',
 'infrastructure',
 'injury',
 'insulation',
 'insurance',
 'integrity',
 'isolation',
 'kitchen',
 'liable',
 'life',
 'maintain',
 'maintenance',
 'medical',
 'neatness',
 'negative',
 'old',
 'outdoors',
 'passion',
 'police',
 'pool',
 'prejudice',
 'preparedness',
 'pressure',
 'prevention',
 'problem',
 'property',
 'protect',
 'protected',
 'protecting',
 'protection',
 'protective',
 'rain',
 'rebuild',
 'rebuilding',
 'reconstructing',
 'reconstruction',
 'recourse',
 'recovery',
 'refurbish',
 'refurbishing',
 'refurbishment',
 'rehabilitation',
 'reliability',
 'remedial',
 'remediate',
 'remediation',
 'remedies',
 'remedy',
 'remedying',
 'renovate',
 'renovation',
 'replace',
 'restoration',
 'restorative',
 'restore',
 'restructuring',
 'risk',
 'risks',
 'risky',
 'road',
 'roof',
 'room',
 'safe',
 'safeguard',
 'safeguarding',
 'safely',
 'safer',
 'safety',
 'sanitary',
 'secure',
 'securing',
 'security',
 'serious',
 'severe',
 'sewerage',
 'space',
 'stable',
 'stairs',
 'steam',
 'stone',
 'structure',
 'structures',
 'sure',
 'surety',
 'tank',
 'tap',
 'temperature',
 'threat',
 'threatening',
 'threats',
 'tidiness',
 'toilet',
 'toilets',
 'traffic',
 'transport',
 'trouble',
 'uncertainty',
 'undesirable',
 'unforeseen',
 'unhealthy',
 'unintentional',
 'unsafe',
 'urgency',
 'ventilation',
 'vigilance',
 'vulnerable',
 'warm',
 'warning',
 'waste',
 'water',
 'weather',
 'wellbeing',
 'wet',
 'wind',
 'winter',
 'worry',
 'yard'}

In [133]:
copy = safety_collection.copy()
for word in list(copy):
    if input('Keep '+ word + '? Press Enter to keep. Press any key + Enter to remove : '):
        safety_collection.remove(word)

Keep hazards? Press Enter to keep. Press any key + Enter to remove : 
Keep floors? Press Enter to keep. Press any key + Enter to remove : n
Keep accident? Press Enter to keep. Press any key + Enter to remove : 
Keep serious? Press Enter to keep. Press any key + Enter to remove : n
Keep ice? Press Enter to keep. Press any key + Enter to remove : n
Keep warning? Press Enter to keep. Press any key + Enter to remove : 
Keep fluid? Press Enter to keep. Press any key + Enter to remove : n
Keep drink? Press Enter to keep. Press any key + Enter to remove : n
Keep maintain? Press Enter to keep. Press any key + Enter to remove : n
Keep refurbishing? Press Enter to keep. Press any key + Enter to remove : n
Keep healthcare? Press Enter to keep. Press any key + Enter to remove : n
Keep pool? Press Enter to keep. Press any key + Enter to remove : n
Keep safer? Press Enter to keep. Press any key + Enter to remove : 
Keep guard? Press Enter to keep. Press any key + Enter to remove : 
Keep security? Pr

Keep reconstructing? Press Enter to keep. Press any key + Enter to remove : n
Keep consolidation? Press Enter to keep. Press any key + Enter to remove : n
Keep protective? Press Enter to keep. Press any key + Enter to remove : 
Keep air? Press Enter to keep. Press any key + Enter to remove : n
Keep tidiness? Press Enter to keep. Press any key + Enter to remove : 
Keep stairs? Press Enter to keep. Press any key + Enter to remove : n
Keep freeze? Press Enter to keep. Press any key + Enter to remove : n
Keep incident? Press Enter to keep. Press any key + Enter to remove : 
Keep protected? Press Enter to keep. Press any key + Enter to remove : n
Keep rain? Press Enter to keep. Press any key + Enter to remove : n
Keep attack? Press Enter to keep. Press any key + Enter to remove : 
Keep hot? Press Enter to keep. Press any key + Enter to remove : n
Keep preparedness? Press Enter to keep. Press any key + Enter to remove : n
Keep buildings? Press Enter to keep. Press any key + Enter to remove :

In [135]:
safety_collection.remove('sure')

In [136]:
safety_collection

{'accident',
 'attack',
 'blast',
 'clean',
 'cleaner',
 'cleaners',
 'cleaning',
 'cleanliness',
 'crisis',
 'damage',
 'danger',
 'dangerous',
 'dangers',
 'defence',
 'devastating',
 'emergency',
 'exposure',
 'fear',
 'fire',
 'guard',
 'harm',
 'hazard',
 'hazardous',
 'hazards',
 'health',
 'healthy',
 'hygiene',
 'hygienic',
 'ill',
 'incident',
 'incidents',
 'injury',
 'maintenance',
 'neatness',
 'police',
 'prevention',
 'protect',
 'protection',
 'protective',
 'rehabilitation',
 'remedial',
 'risk',
 'risks',
 'risky',
 'safe',
 'safeguard',
 'safeguarding',
 'safely',
 'safer',
 'safety',
 'sanitary',
 'secure',
 'security',
 'surety',
 'threat',
 'threatening',
 'threats',
 'tidiness',
 'unhealthy',
 'unsafe',
 'urgency',
 'vigilance',
 'vulnerable',
 'warning'}

In [137]:
building_collection = {'accident',
 'adverse',
 'aerial',
 'air',
 'architect',
 'architectural',
 'architecture',
 'assurance',
 'assured',
 'atmosphere',
 'attack',
 'basin',
 'bathroom',
 'benign',
 'blast',
 'block',
 'breath',
 'bucket',
 'building',
 'buildings',
 'built',
 'car',
 'clean',
 'cleaner',
 'cleaners',
 'cleaning',
 'cleanliness',
 'comfort',
 'compromise',
 'concern',
 'conservation',
 'consolidate',
 'consolidating',
 'consolidation',
 'construct',
 'constructed',
 'constructing',
 'construction',
 'constructions',
 'cool',
 'crisis',
 'damage',
 'danger',
 'dangerous',
 'dangers',
 'defence',
 'depletion',
 'detriment',
 'devastating',
 'difficulty',
 'disruptive',
 'dog',
 'drainage',
 'drains',
 'drink',
 'emergency',
 'engineering',
 'environmental',
 'error',
 'exposure',
 'facility',
 'fear',
 'fire',
 'fixed',
 'flood',
 'flooding',
 'floor',
 'floors',
 'fluid',
 'food',
 'foyer',
 'freeze',
 'gas',
 'guarantee',
 'guaranteed',
 'guarantees',
 'guard',
 'guarding',
 'habitat',
 'hall',
 'hard',
 'harm',
 'hazard',
 'hazardous',
 'hazards',
 'health',
 'healthcare',
 'healthy',
 'hot',
 'hygiene',
 'hygienic',
 'ice',
 'ill',
 'incidence',
 'incident',
 'incidents',
 'infrastructure',
 'injury',
 'insulation',
 'insurance',
 'integrity',
 'isolation',
 'kitchen',
 'liable',
 'life',
 'maintain',
 'maintenance',
 'medical',
 'neatness',
 'negative',
 'old',
 'outdoors',
 'passion',
 'police',
 'pool',
 'prejudice',
 'preparedness',
 'pressure',
 'prevention',
 'problem',
 'property',
 'protect',
 'protected',
 'protecting',
 'protection',
 'protective',
 'rain',
 'rebuild',
 'rebuilding',
 'reconstructing',
 'reconstruction',
 'recourse',
 'recovery',
 'refurbish',
 'refurbishing',
 'refurbishment',
 'rehabilitation',
 'reliability',
 'remedial',
 'remediate',
 'remediation',
 'remedies',
 'remedy',
 'remedying',
 'renovate',
 'renovation',
 'replace',
 'restoration',
 'restorative',
 'restore',
 'restructuring',
 'risk',
 'risks',
 'risky',
 'road',
 'roof',
 'room',
 'safe',
 'safeguard',
 'safeguarding',
 'safely',
 'safer',
 'safety',
 'sanitary',
 'secure',
 'securing',
 'security',
 'serious',
 'severe',
 'sewerage',
 'space',
 'stable',
 'stairs',
 'steam',
 'stone',
 'structure',
 'structures',
 'sure',
 'surety',
 'tank',
 'tap',
 'temperature',
 'threat',
 'threatening',
 'threats',
 'tidiness',
 'toilet',
 'toilets',
 'traffic',
 'transport',
 'trouble',
 'uncertainty',
 'undesirable',
 'unforeseen',
 'unhealthy',
 'unintentional',
 'unsafe',
 'urgency',
 'ventilation',
 'vigilance',
 'vulnerable',
 'warm',
 'warning',
 'waste',
 'water',
 'weather',
 'wellbeing',
 'wet',
 'wind',
 'winter',
 'worry',
 'yard'}